In [1]:
import pandas as pd
movie_data = pd.read_csv('https://raw.githubusercontent.com/JonatasDeOliveira/movie-it/master/src/data/moviesOnStreamingPlatforms_updated.csv',sep=',')

In [2]:
movie_data.head()

,Unnamed: 0,ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Type,Directors,Genres,Country,Language,Runtime
0,0,1,Inception,2010,13+,8.8,87%,1,0,0,0,0,Christopher Nolan,"Action,Adventure,Sci-Fi,Thriller","United States,United Kingdom","English,Japanese,French",148.0
1,1,2,The Matrix,1999,18+,8.7,87%,1,0,0,0,0,"Lana Wachowski,Lilly Wachowski","Action,Sci-Fi",United States,English,136.0
2,2,3,Avengers: Infinity War,2018,13+,8.5,84%,1,0,0,0,0,"Anthony Russo,Joe Russo","Action,Adventure,Sci-Fi",United States,English,149.0
3,3,4,Back to the Future,1985,7+,8.5,96%,1,0,0,0,0,Robert Zemeckis,"Adventure,Comedy,Sci-Fi",United States,English,116.0
4,4,5,"The Good, the Bad and the Ugly",1966,18+,8.8,97%,1,0,1,0,0,Sergio Leone,Western,"Italy,Spain,West Germany",Italian,161.0


Queremos agora analisar os tipos dos dados da tabela e modificar caso necessário

In [3]:
movie_data.dtypes

Unnamed: 0           int64
ID                   int64
Title               object
Year                 int64
Age                 object
IMDb               float64
Rotten Tomatoes     object
Netflix              int64
Hulu                 int64
Prime Video          int64
Disney+              int64
Type                 int64
Directors           object
Genres              object
Country             object
Language            object
Runtime            float64
dtype: object

### Install PySpark

In [4]:
!pip install pyspark

     |████████████████████████████████| 204.2MB 67kB/s 
     |████████████████████████████████| 204kB 42.9MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=37c997e4234db05fe866dfcac889a7a70ba4b89625a66598a8745da948b666a7
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


### Create Spark Context

In [6]:
from pyspark import SparkContext
from pyspark.sql import Row
from pyspark.sql import *
from pyspark.sql import SQLContext
from pyspark import SparkFiles

In [7]:
spark = SparkContext("local", "MovieIt App")

In [8]:
spark.addFile('https://raw.githubusercontent.com/JonatasDeOliveira/movie-it/master/src/data/moviesOnStreamingPlatforms_updated.csv')

In [9]:
sql_context = SQLContext(spark)
movies_df = sql_context.read.csv(SparkFiles.get("moviesOnStreamingPlatforms_updated.csv"), header=True, inferSchema= True)

In [10]:
movies_df.show(20, False)

+---+---+----------------------------------+----+---+----+---------------+-------+----+-----------+-------+----+-------------------------------------------+----------------------------------------+-------------------------------------------+-------------------------------------------+-------+
|_c0|ID |Title                             |Year|Age|IMDb|Rotten Tomatoes|Netflix|Hulu|Prime Video|Disney+|Type|Directors                                  |Genres                                  |Country                                    |Language                                   |Runtime|
+---+---+----------------------------------+----+---+----+---------------+-------+----+-----------+-------+----+-------------------------------------------+----------------------------------------+-------------------------------------------+-------------------------------------------+-------+
|0  |1  |Inception                         |2010|13+|8.8 |87%            |1      |0   |0          |0      |0   |Christ

In [11]:
movies_df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- ID: integer (nullable = true)
 |-- Title: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- IMDb: double (nullable = true)
 |-- Rotten Tomatoes: string (nullable = true)
 |-- Netflix: integer (nullable = true)
 |-- Hulu: integer (nullable = true)
 |-- Prime Video: integer (nullable = true)
 |-- Disney+: integer (nullable = true)
 |-- Type: integer (nullable = true)
 |-- Directors: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Language: string (nullable = true)
 |-- Runtime: integer (nullable = true)



Pontos notados:

- Primeiramente, percebemos que temos duas colunas com "ids". Como pegamos esses dados do kaggle possivelmente esta primeira coluna não é necessária.
- Outra observação é que as colunas Directors, Genres, Country e Language são listas e estão como strings devemos processar também as mesmas.
- Colunas Age e Year estão como string, devem ser transformadas para integer.

In [12]:
from pyspark.sql.functions import split
from pyspark.sql.functions import col

In [13]:
movies_df_dropc0 = movies_df.drop('_c0')

In [14]:
movies_df_split = (movies_df_dropc0.withColumn('DirectorsArray', split('Directors', ','))
.withColumn('GenresArray', split('Genres', ','))
.withColumn('CountriesArray', split('Country', ','))
.withColumn('LanguagesArray', split('Language', ','))
.drop('Directors')
.drop('Genres')
.drop('Country')
.drop('Language'))

In [15]:
# checando se todas as idades seguem o regex [0-9]+\+
df_filtered_age_pattern = movies_df_split.filter(col('Age').rlike('[0-9]+\+'))
df_filtered_age_pattern.count() == movies_df_split.count()

False

In [16]:
# verificar idades que não seguem o padrão
df_filtered_not_age_pattern = movies_df_split.filter(~col('Age').rlike('[0-9]+\+'))
df_filtered_not_age_pattern.show(5, False)

+---+-----------------------------------+----+---+----+---------------+-------+----+-----------+-------+----+-------+---------------------------+-----------------------------------------------------------------+----------------------------------------------------+----------------------------------+
|ID |Title                              |Year|Age|IMDb|Rotten Tomatoes|Netflix|Hulu|Prime Video|Disney+|Type|Runtime|DirectorsArray             |GenresArray                                                      |CountriesArray                                      |LanguagesArray                    |
+---+-----------------------------------+----+---+----+---------------+-------+----+-----------+-------+----+-------+---------------------------+-----------------------------------------------------------------+----------------------------------------------------+----------------------------------+
|55 |Willy Wonka & the Chocolate Factory|1971|all|7.8 |91%            |1      |0   |0          |0   

In [18]:
# verficar que o padrão de idade em regex [0-9]+\+ unido a o padrão com 'all' contemplam todo o dataframe
df_filtered_pattern_all = movies_df_split.filter(col('Age').rlike('[0-9]+\+') | (col('Age') == 'all'))
movies_df_split.count() == df_filtered_pattern_all.count()

False

In [20]:
df_filtered_not_pattern_all = movies_df_split.filter(~(col('Age').rlike('[0-9]+\+') | (col('Age') == 'all')))
df_filtered_not_pattern_all.show(5, False)

+-----+-----------------------------+-------------------------+----+----+---------------+-------+----+-----------+-------+----+-------+--------------+-----------+--------------+--------------+
|ID   |Title                        |Year                     |Age |IMDb|Rotten Tomatoes|Netflix|Hulu|Prime Video|Disney+|Type|Runtime|DirectorsArray|GenresArray|CountriesArray|LanguagesArray|
+-----+-----------------------------+-------------------------+----+----+---------------+-------+----+-----------+-------+----+-------+--------------+-----------+--------------+--------------+
|15902|"""22"" A film about Veterans| Healthcare and Suicide."|2018|null|null           |null   |0   |0          |1      |0   |null   |[0]           |null       |null          |null          |
+-----+-----------------------------+-------------------------+----+----+---------------+-------+----+-----------+-------+----+-------+--------------+-----------+--------------+--------------+



Claramente a linha apresentada acima é um erro pois a idade está como um ano, o ano está como um texto e a maioria das outras colunas estão vazias, vamos remover essa linha.

In [30]:
movies_df_filtered_age = movies_df_split.where(col('ID') != 15902)

Agora vamos processar Age para aparecer como um inteiro em todo dataframe

In [27]:
movies_df_filtered_age.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Title: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- IMDb: double (nullable = true)
 |-- Rotten Tomatoes: string (nullable = true)
 |-- Netflix: integer (nullable = true)
 |-- Hulu: integer (nullable = true)
 |-- Prime Video: integer (nullable = true)
 |-- Disney+: integer (nullable = true)
 |-- Type: integer (nullable = true)
 |-- Runtime: integer (nullable = true)
 |-- DirectorsArray: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- GenresArray: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- CountriesArray: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- LanguagesArray: array (nullable = true)
 |    |-- element: string (containsNull = true)



Analisando o schema novamente, podemos ver que assim como o campo Age, o campo Year deveria estar como Integer, vamos arrumar isso

In [33]:
from pyspark.sql.types import IntegerType
movies_df_filtered = movies_df_filtered_age.withColumn("Age", movies_df_filtered_age["Age"].cast(IntegerType()))
movies_df_filtered = movies_df_filtered.withColumn("Year", movies_df_filtered["Year"].cast(IntegerType()))

In [34]:
movies_df_filtered.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Title: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Age: integer (nullable = true)
 |-- IMDb: double (nullable = true)
 |-- Rotten Tomatoes: string (nullable = true)
 |-- Netflix: integer (nullable = true)
 |-- Hulu: integer (nullable = true)
 |-- Prime Video: integer (nullable = true)
 |-- Disney+: integer (nullable = true)
 |-- Type: integer (nullable = true)
 |-- Runtime: integer (nullable = true)
 |-- DirectorsArray: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- GenresArray: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- CountriesArray: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- LanguagesArray: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [47]:
movies_df_filtered.orderBy('IMDb', ascending=False).show(1, False)

+----+---------------+----+----+----+---------------+-------+----+-----------+-------+----+-------+--------------+---------------------------------+---------------+--------------+
|ID  |Title          |Year|Age |IMDb|Rotten Tomatoes|Netflix|Hulu|Prime Video|Disney+|Type|Runtime|DirectorsArray|GenresArray                      |CountriesArray |LanguagesArray|
+----+---------------+----+----+----+---------------+-------+----+-----------+-------+----+-------+--------------+---------------------------------+---------------+--------------+
|5111|Love on a Leash|2011|null|9.3 |null           |0      |0   |1          |0      |0   |90     |[Fen Tian]    |[Comedy, Drama, Fantasy, Romance]|[United States]|null          |
+----+---------------+----+----+----+---------------+-------+----+-----------+-------+----+-------+--------------+---------------------------------+---------------+--------------+
only showing top 1 row



In [50]:
movies_df_filtered.where((col("Netflix") == 1) & (col("Prime Video") == 1)).orderBy('IMDb', ascending=False).show(1, False)

+---+------------------------------+----+----+----+---------------+-------+----+-----------+-------+----+-------+--------------+-----------+----------------------------+--------------+
|ID |Title                         |Year|Age |IMDb|Rotten Tomatoes|Netflix|Hulu|Prime Video|Disney+|Type|Runtime|DirectorsArray|GenresArray|CountriesArray              |LanguagesArray|
+---+------------------------------+----+----+----+---------------+-------+----+-----------+-------+----+-------+--------------+-----------+----------------------------+--------------+
|5  |The Good, the Bad and the Ugly|1966|null|8.8 |97%            |1      |0   |1          |0      |0   |161    |[Sergio Leone]|[Western]  |[Italy, Spain, West Germany]|[Italian]     |
+---+------------------------------+----+----+----+---------------+-------+----+-----------+-------+----+-------+--------------+-----------+----------------------------+--------------+
only showing top 1 row



In [51]:
movies_df_filtered.where((col("Netflix") == 1) & (col("Prime Video") == 1)).orderBy('Rotten Tomatoes', ascending=False).show(1, False)

+---+----------+----+----+----+---------------+-------+----+-----------+-------+----+-------+-----------------+--------------+----------------+--------------+
|ID |Title     |Year|Age |IMDb|Rotten Tomatoes|Netflix|Hulu|Prime Video|Disney+|Type|Runtime|DirectorsArray   |GenresArray   |CountriesArray  |LanguagesArray|
+---+----------+----+----+----+---------------+-------+----+-----------+-------+----+-------+-----------------+--------------+----------------+--------------+
|143|Starred Up|2013|null|7.4 |99%            |1      |0   |1          |0      |0   |106    |[David Mackenzie]|[Crime, Drama]|[United Kingdom]|[English]     |
+---+----------+----+----+----+---------------+-------+----+-----------+-------+----+-------+-----------------+--------------+----------------+--------------+
only showing top 1 row

